In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import tensorflow as tf
import glob
import os
import time
import pims
import imageio
from scipy import misc
imageio.plugins.ffmpeg.download()
from moviepy.editor import *

def byte_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# def get_label(name):
#     n, _ = os.path.splitext(name)
#     return int(n.split("_")[-1])-1
labels = ["get", "pick", "kick", "punch", "wave", "sit", "scratch", "cross", "turn", "check", "point", "walk"]
# def get_label(name):
#     n, _ = os.path.splitext(name)
#     n = n.split("/")[-1]
#     n = n.split("_")[2]
#     n = n.split("-")[0]
    
#     return int(labels.index(n))

def get_label(name):
    n, _ = os.path.splitext(name)
    n = n.split("/")[-1]
    n = n.split("_")[-1]
    
    
    return int(n) -1 

def video_to_tensor(video, h=128, w=128, depth = 9):
    num = 0
    try:
        images = pims.Video(video)
        num = len(images)
#         ratio = float(num) / depth if num > depth else 1.0
#         if num <= 0:
#             return [], None
#         im_shape = images[0].shape
        tensor = np.ndarray((h,w,depth))
#         img_prev = images[0]
#         img_prev = (0.2989 * img_prev[:,:,0] + 0.5870 * img_prev[:,:,1] + 0.1140 * img_prev[:,:,2])
#         for i in xrange(1,depth):
# #             index = int(round(ratio*i))
# #             if(index<num):
#             img_color = images[index]
# #                 img_gray = (0.2989 * img_color[:,:,0] + 0.5870 * img_color[:,:,1] + 0.1140 * img_color[:,:,2])
                
# #                 image = misc.imresize(img_gray,(w,h)) #np.abs(img_prev - img_gray)
                
# #                 img_prev = img_gray
                
                
# #                 image  = np.transpose(img, (2, 0, 1))
#             tensor[:,:,i]=image
    
    
        for i in range(3):
            tensor[:,:,i*3:i*3+3]=images[i]
        
        del images
        label = get_label(video)
       
        return tensor.astype(np.uint8), label
    except:
        print("erro ao carregar:", video, "tam:", num)
        return [], None
        

In [3]:
# import shutil as s
# videos_path_train = '../../datasets/IXMAS'
# def rep(src, dest):
#     dest = src.replace("IXMAS",dest)
#     s.copy2(src,dest)
    
# files = glob.glob(os.path.join(videos_path_train, '*.avi'))
# print("total:", len(files))
# np.random.shuffle(files)
# size = int(len(files)*0.3)
# test = files[:size]
# train = files[size:]
# map(rep,test,["ixmas_test"]*len(test))
# map(rep,train,["ixmas_train"]*len(train))
# print('total images:',len(train))

In [4]:
9# all_labels = ['prendere', 'vieniqui', 'perfetto', 'fame', 'sonostufo', 'seipazzo', 'basta', 'cheduepalle', 'noncenepiu', 'chevuoi',
#               'ok', 'combinato', 'freganiente', 'cosatifarei', 'buonissimo', 'vattene', 'messidaccordo', 'daccordo', 'furbo', 'tantotempo']

record_file_path = './video_tfrecord/star_test_rgb.tfrecords'
videos_path_train = './datasets/test_star_RGB'

files = glob.glob(os.path.join(videos_path_train, '*'))

np.random.shuffle(files)
print('total images:',len(files))


# filename_queue = tf.train.string_input_producer(files, shuffle=False, num_epochs=1)
writer = tf.python_io.TFRecordWriter(record_file_path)
reader = tf.WholeFileReader()

with tf.device('/cpu:0'):
#     key, value = reader.read(filename_queue)
#     decoded_image = tf.image.decode_png(value) # tf.image.decode_image
#     decoded_image = tf.reshape(tf.cast(decoded_image, tf.uint8), [-1])
    t = time.time()
    count = 0
    config = tf.ConfigProto()
    config.intra_op_parallelism_threads = 8
    config.inter_op_parallelism_threads = 8

    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        for video in files:
           
            tensor, label = video_to_tensor(video, depth=9)
            
            if label == None:
                continue
                
            example = tf.train.Example(features=tf.train.Features(feature={       
                'video_raw': byte_feature(tensor.tostring()), 
                'label':int_feature([label])
            }))
            writer.write(example.SerializeToString())
            if count % 50 == 0:
                print('round:', count)
                
            count += 1
            del tensor

        writer.close()
        coord.request_stop()
        coord.join(threads)
    print('elapsed time:',time.time() - t)

total images: 3579
round: 0
round: 50
round: 100
round: 150
round: 200
round: 250
round: 300
round: 350
round: 400
round: 450
round: 500
round: 550
round: 600
round: 650
round: 700
round: 750
round: 800
round: 850
round: 900
round: 950
round: 1000
round: 1050
round: 1100
round: 1150
round: 1200
round: 1250
round: 1300
round: 1350
round: 1400
round: 1450
round: 1500
round: 1550
round: 1600
round: 1650
round: 1700
round: 1750
round: 1800
round: 1850
round: 1900
round: 1950
round: 2000
round: 2050
round: 2100
round: 2150
round: 2200
round: 2250
round: 2300
round: 2350
round: 2400
round: 2450
round: 2500
round: 2550
round: 2600
round: 2650
round: 2700
round: 2750
round: 2800
round: 2850
round: 2900
round: 2950
round: 3000
round: 3050
round: 3100
round: 3150
round: 3200
round: 3250
round: 3300
round: 3350
round: 3400
round: 3450
round: 3500
round: 3550
elapsed time: 940.770774841


In [5]:
# videos_path = '../../datasets/train_split'

# with tf.device('/cpu:0'):
#     max, min = 0, 1000
#     sum, qtd = 0, 0
#     for video in glob.glob(os.path.join(videos_path, '*')):
#         name=  video.split("-")
#         beg= int(name[-3])
#         end = int(name[-2])
#         n = end - beg + 1
#         if n >40:  qtd += 1
#         sum += n
       
#         if(max < n): max = n
#         if(min>n): min = n
#     media = sum / qtd

#     print(max, min, qtd)

    

In [6]:
# import math
# t = 2.6
# round(t)